# Rauk's table and Wolfsberrg-Helmholz approximation


## Introduction

So far the Hamiltonians treated by the documentation have been focused in carbon chains. With this new update it is possible to work with different atoms, using the Rauk's table or the distance beetween the atoms with the Wolfsberrg-Helmholz aproximation.

To generate a Hamiltonian with atoms different from carbon, simply apply the same logic as before: Define a list of tuples with the atoms indexed by the position of their respective site in the compound. The new module of the library has two main purposes:

1) If you define the system using atoms different from carbons and the connectivity as ``integer`` numbers, the program will assign to the one body term the $\alpha$ and $\beta$ precomputed based on the Rauk's table.

2) If you define the system using atoms different from carbons and the connectivity as `float` numbers, the program will assign to the one body term the $\alpha$ and $\beta$ computed using the Wolfsberrg-Helmholz approximation, based on the provided distance.



## Example: Rauk's table

Suppose that you want to define a Hubbard model assigning the values based on the Rauk's table (RAUK, 2001). The calculations of $\alpha$ and $\beta$ were made based on Simple Hückel Molecular Orbital Theory(SHMO), following the following equations, for the atoms X, Y:

$$
\alpha_X = \alpha h_x|\beta| 
$$
$$
b_{XY} =  \kappa_{XY}|\beta|
$$

‌

You can do this based on the example below. 




In [19]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub

# Define the molecular system as a list of tuples.
# The format is ('Atom1', 'Atom2', bond_order).
# Here, 'Atom1' and 'Atom2' are the atomic symbols of the atoms involved in the bond.
# 'bond_order' is the order of the bond between the atoms.
# For example, ('C1', 'Cl2', 1) represents a single bond between a carbon atom and a chlorine atom.
system = [('C1', 'Cl2', 1), ('Cl2', 'F3', 1), ('F3', 'Si4', 1), ('Si4', 'C1', 1)]

# Create an instance of the HamHub class, which represents the Hamiltonian for the system.
# Here, 'u_onsite' is an array specifying the on-site interaction energies for the atoms.
# The length of 'u_onsite' should match the number of distinct atoms in the system.
hubbard = HamHub(system, u_onsite=np.array([1, 1]))

# Generate the one-body integral matrix in the specified basis.
# 'dense=True' indicates that the integral matrix should be returned in a dense format.
# 'basis' specifies the type of basis to use, here it's 'spatial basis'.
print(hubbard.generate_one_body_integral(dense=True, basis='spatial basis'))


[[-0.414    -0.033046  0.       -0.039975]
 [-0.033046 -0.492884 -0.027183  0.      ]
 [ 0.       -0.027183 -0.558443 -0.009061]
 [-0.039975  0.       -0.009061 -0.414   ]]


## Example:  Wolfsberrg-Helmholz approximation

Suppose now that you want to define the same system, but computing the parameters based on the overlap distance beetween the neighbors atoms. You can do that using the example below.

The values of $\alpha$ and $\beta$ are computed based on the following formulas:


$$
\alpha_X = -(\text{ionization potential of atom X})
$$

The $\beta$ value for the interaction between atom X and atom Y is defined as:

$$
\beta_{XY} = 1.75 S_{XY} \frac{\alpha_X + \alpha_Y}{2}
$$



In [20]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub

# Define the molecular system as a list of tuples.
# Each tuple represents a bond between two atoms, the bond length, and the bond type.
# The bond length is a float value representing the distance between the atoms.
# The bond type can be either 'sigma' or 'pi'.
# For example, ('C1', 'Cl2', 1.5, 'pi') represents a pi bond between a carbon atom and a chlorine atom.
system = [
    ('C1', 'Cl2', 1.5, 'pi'), 
    ('Cl2', 'F3', 1.8, 'sigma'),
    ('F3', 'Si4', 1.8, 'sigma'), 
    ('Si4', 'C1', 1.7, 'sigma')
]

# As before, create an instance of the HamHub class, which represents the Hamiltonian for the system.
hubbard = HamHub(system, u_onsite=np.array([1, 1]))

# Generate the one-body integral matrix in the specified basis.
# 'dense=True' indicates that the integral matrix should be returned in a dense format.
# 'basis' specifies the type of basis to use, here it's 'spatial basis'.
print(hubbard.generate_one_body_integral(dense=True, basis='spatial basis'))


[[-0.41380839 -0.73648465  0.         -0.56689357]
 [-0.73648465 -0.47655051 -0.81351185  0.        ]
 [ 0.         -0.81351185 -0.64027609 -0.60447297]
 [-0.56689357  0.         -0.60447297 -0.29956945]]


# Example: Usage with molecules 

If you want to make a chain using molecules you can use the following structure: pass the atom $X_n$ in the position m, the tuple would have the following form `Xm(n)`. Below it's possible to see a example of this of the molecule $N_2OP_2$. 

In [21]:
from moha import HamHuck

# Define the molecular system with tuples in the format
# ('Atomposition(index)', 'Neighboorposition(index)', 'bond_order').
# Each tuple represents a bond between atoms at specified positions.
system =[('N1(2)', 'O2(1)', 1),
         ('O2(1)', 'P3(2)', 1),
         ('P3(2)', 'N1(2)', 1)]

# Create an instance of HamHuck with the defined system.
N2_O1_P2 = HamHuck(system)

# Generate the one-body integral matrix in the 'spatial basis'.
h1 = N2_O1_P2.generate_one_body_integral(dense=True, basis='spatial basis')

# Print the one-body integral matrix.
print(h1)

[[-0.441183 -0.060762 -0.041574]
 [-0.060762 -0.465701 -0.039975]
 [-0.041574 -0.039975 -0.424127]]


## Example: Defining Your Own Dictionaries

Instead of using Rauk's Table or the Wolfsberg-Helmholz approximation, you can define your own dictionaries for $\alpha$ and $\beta$ values. These dictionaries specify the interaction parameters for each atom and bond in your system.

To do this, pass the dictionaries to the Hamiltonian classes with the following structure:

1. `atom_dictionary = {'atom1': alpha1, 'atom2': alpha2}`
2. `bond_dictionary = {'atom1,atom2': beta}`


In [22]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub

# Define the Hamiltonian for a molecular system using the HamHub class.

# The system is a list of tuples, where each tuple represents a bond between two atoms.
# The format of each tuple is ('Atom1', 'Atom2', bond_order).
# For this example, we define single bonds between different atoms.
system = [
    ('C1', 'Cl2', 1),  # Bond between C1 and Cl2
    ('Cl2', 'F3', 1),  # Bond between Cl2 and F3
    ('F3', 'Si4', 1),  # Bond between F3 and Si4
    ('Si4', 'C1', 1)   # Bond between Si4 and C1
]

# Create an instance of the HamHub class, which represents the Hamiltonian for the system.
# 'u_onsite' is an array specifying the on-site interaction energies for the atoms.
# 'atom_dictionary' maps atom symbols to unique identifiers.
# 'bond_dictionary' maps bond pairs to unique identifiers.
hubbard = HamHub(system, u_onsite=np.array([1, 1]), 
                 atom_dictionary={'C': 1, 'Cl': 2, 'F': 3, 'Si': 4},
                 bond_dictionary={'C,Cl': 0, 'Cl,F': 1, 'F,Si': 2, 'Si,C': 3})

# Generate the one-body integral matrix in the specified basis.
# 'dense=True' indicates that the integral matrix should be returned in a dense format.
# 'basis' specifies the type of basis to use, here it's 'spatial basis'.
print(hubbard.generate_one_body_integral(dense=True, basis='spatial basis'))


[[1. 0. 0. 3.]
 [0. 2. 1. 0.]
 [0. 1. 3. 2.]
 [3. 0. 2. 4.]]


## Example: Second Body Terms - Pariser-Parr Module

The software is highly flexible in generating the two-body terms.

If the user does not provide `u_onsite` or `affinity_dct`, the potentials are computed using the following formula: $ U_x = \alpha_x - A_x $.

If `orbital_overlap`is not provided, the values of `u_onsite`, either provided or computed, are used to calculate the potential between two sites:

$$
\overline{U}_{XY} = \frac{1}{2}(U_X + U_Y)
$$

If the value of `gamma` is not provided, it is computed using the formula:

$$
\gamma_{XY} = \frac{\overline{U}_{XY}}{\overline{U}_{XY} R_{XY} + e^{-\frac{1}{2} \overline{U}_{XY}^2 R_{XY}^2}}
$$

In [23]:
import sys  
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub

# Example: Generating XXZ Heisenberg model 
# Returning electron integrals in a spatial orbital basis
# Assuming 4-fold symmetry
# Returning output as dense matrix

# Define the molecular system with tuples in the format ('Atom1', 'Atom2', bond_length).
# The system represents bonds between atoms with specified bond lengths.
system = [
    ('C1', 'Cl2', 1.1),  # Bond between C1 and Cl2 with bond length 1.1
    ('Cl2', 'F3', 1.0),  # Bond between Cl2 and F3 with bond length 1.0
    ('F3', 'Si4', 1.0),  # Bond between F3 and Si4 with bond length 1.0
    ('Si4', 'C1', 1.0)   # Bond between Si4 and C1 with bond length 1.0
]

# Create an instance of the HamHub class with the defined system.
# u_onsite specifies the on-site interaction energies for the atoms.
# orbital_overlap specifies the overlap integrals between orbitals.
# So the gamma will be computed based on orbital_overlap.
ham = HamHub(system, u_onsite=np.array([1, 1, 1, 1]),
                 orbital_overlap=np.array([[0, 1, 0, 0],[0, 0, 3, 0], [0, 0, 0, 4], [0, 0, 0, 1]]))

# Generate the zero-body integral.
e0 = ham.generate_zero_body_integral()

# Generate the one-body integral matrix in the spatial orbital basis.
h1 = ham.generate_one_body_integral(dense=True, basis='spatial basis')

# Generate the two-body integral tensor in the spatial orbital basis, assuming 4-fold symmetry.
h2 = ham.generate_two_body_integral(dense=True, basis='spatial basis', sym=4)

print("Zero energy: ", e0)
print("One body integrals in spatial basis: \n", h1)
print("Shape of two body integral in spatial basis: ", h2.shape)
print("-" * 60)

# Example: Generating XXZ Heisenberg model in spin orbital basis
# Assuming 4-fold symmetry
# Returning output as dense matrix

# Generate the one-body integral matrix in the spin orbital basis.
h1 = ham.generate_one_body_integral(dense=True, basis='spinorbital basis')

# Generate the two-body integral tensor in the spin orbital basis, assuming 4-fold symmetry.
h2 = ham.generate_two_body_integral(dense=True, basis='spinorbital basis', sym=4)

print("One body integrals in spinorbital basis: \n", h1)
print("Shape of two body integral in spinorbital basis: ", h2.shape)


Zero energy:  0
One body integrals in spatial basis: 
 [[-0.41380839 -0.77906404  0.          0.        ]
 [-0.77906404 -0.47655051 -2.93166983  0.        ]
 [ 0.         -2.93166983 -0.64027609 -3.28945939]
 [ 0.          0.         -3.28945939 -0.29956945]]
Shape of two body integral in spatial basis:  (4, 4, 4, 4)
------------------------------------------------------------
One body integrals in spinorbital basis: 
 [[-0.41380839 -0.77906404  0.          0.          0.          0.
   0.          0.        ]
 [-0.77906404 -0.47655051 -2.93166983  0.          0.          0.
   0.          0.        ]
 [ 0.         -2.93166983 -0.64027609 -3.28945939  0.          0.
   0.          0.        ]
 [ 0.          0.         -3.28945939 -0.29956945  0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.41380839 -0.77906404
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.77906404 -0.47655051
  -2.93166983  0.        ]
 [ 

In [24]:
import sys
sys.path.insert(0, '../')
import numpy as np
from moha import HamHub

# Example: Generating 6-site Hubbard model 
# Returning electron integrals in a spatial orbital basis
# Assuming 8-fold symmetry
# Returning output as dense matrix

# Define the connectivity matrix for the 6-site Hubbard model.
# Each entry in the matrix indicates the connection between sites.
connectivity = np.array([
    [0, 1, 0, 0, 0, 1],  
    [1, 0, 1, 0, 0, 0],  
    [0, 1, 0, 1, 0, 0],  
    [0, 0, 1, 0, 1, 0],  
    [0, 0, 0, 1, 0, 1],  
    [1, 0, 0, 0, 1, 0]   
])

# Create an instance of the HamHub class with the defined connectivity.
# alpha and beta are interaction parameters.
# u_onsite specifies the on-site interaction energies for the sites.
hubbard = HamHub(connectivity, alpha=0, beta=-1, u_onsite=np.array([1, 1, 1, 1, 1, 1]))

# Generate the zero-body integral.
e0 = hubbard.generate_zero_body_integral()

# Generate the one-body integral matrix in the spatial orbital basis.
h1 = hubbard.generate_one_body_integral(dense=True, basis='spatial basis')

# Once the gamma was not provided it will be computed based on the one-body integrals.
h2 = hubbard.generate_two_body_integral(dense=True, basis='spatial basis', sym=8)

print("Zero energy: ", e0)
print("One body integrals in spatial basis: \n", h1)
print("Shape of two body integral in spatial basis: ", h2.shape)
print("-" * 60)

# Example: Generating Hubbard model in spin orbital basis
# Assuming 8-fold symmetry
# Returning output as dense matrix

# Generate the one-body integral matrix in the spin orbital basis.
h1 = hubbard.generate_one_body_integral(dense=True, basis='spinorbital basis')

# Generate the two-body integral tensor in the spin orbital basis, assuming 8-fold symmetry.
h2 = hubbard.generate_two_body_integral(dense=True, basis='spinorbital basis', sym=8)

print("One body integrals in spinorbital basis: \n", h1)
print("Shape of two body integral in spinorbital basis: ", h2.shape)


Zero energy: 

 0
One body integrals in spatial basis: 
 [[ 0. -1.  0.  0.  0. -1.]
 [-1.  0. -1.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.  0.]
 [ 0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0. -1.  0. -1.]
 [-1.  0.  0.  0. -1.  0.]]
Shape of two body integral in spatial basis:  (6, 6, 6, 6)
------------------------------------------------------------
One body integrals in spinorbital basis: 
 [[ 0. -1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0. -1.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0. -1.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0. -1.  0.]]
Shape of two body i

## References:

[1]A. Orbital Interaction Theory of Organic Chemistry. [s.l.] John Wiley & Sons, 2004.

[2] https://en.wikipedia.org/wiki/Electron_affinity_(data_page)